In [7]:
from vespa.application import Vespa
from vespa.io import VespaQueryResponse
from vespa.exceptions import VespaError

app = Vespa(url="http://localhost:8080")

In [26]:
with app.syncio() as session:
    response: VespaQueryResponse = session.query(
        yql="select title, overview, overview_embedding from film where title contains \"star trek\"",
        hits=1,
    )
    if response.is_successful():
        print("Success!")
        hit = response.json["root"]["children"][0]
        print(f"Title: {hit["fields"]["title"]}")
        print(f"Overview: {hit["fields"]["overview"]}")
        embedding = hit["fields"]["overview_embedding"]["values"]
        print(f"Embedding: {embedding}")

Success!
Title: Star Trek: The Motion Picture
Overview: When a destructive space entity is spotted approaching Earth, Admiral Kirk resumes command of the Starship Enterprise in order to intercept, examine, and hopefully stop it.
Embedding: [0.025553816929459572, 0.012888586148619652, 0.08634045720100403, -0.001496409298852086, 0.061944834887981415, -0.0626501590013504, 0.06186068430542946, -0.044612158089876175, 0.017161650583148003, 0.03508596494793892, 0.01493413932621479, 0.03167730197310448, -0.03439978137612343, -0.04911120980978012, -0.06296917051076889, -0.04877905547618866, 0.00320075755007565, -0.00978163629770279, 0.028037481009960175, 0.007719686254858971, -0.037230461835861206, 0.07503195106983185, 0.024272270500659943, 0.03238018602132797, -0.10772485285997391, -0.02485300973057747, 0.06828740239143372, -0.06182868778705597, -0.07173274457454681, -0.06587034463882446, 0.0019023400964215398, 0.0011038616066798568, 0.01773732341825962, 0.06004824489355087, -0.006239785812795

In [42]:
with app.syncio() as session:
    response: VespaQueryResponse = session.query(
        body={
            "yql": "select title,overview from film where {targetHits: 3}nearestNeighbor(overview_embedding, query_embedding)",
            "ranking": "similarity",
            "input.query(query_embedding)": embedding,
        }
    )
    if response.is_successful():
        print("Success!")
        print(response.json["root"]["children"])

Success!
[{'id': 'index:films/0/37a749d8b9c67a07065d6c90', 'relevance': 1.0, 'source': 'films', 'fields': {'title': 'Star Trek: The Motion Picture', 'overview': 'When a destructive space entity is spotted approaching Earth, Admiral Kirk resumes command of the Starship Enterprise in order to intercept, examine, and hopefully stop it.'}}, {'id': 'index:films/0/d3d0515e168446c7f85cd617', 'relevance': 0.5285139407812043, 'source': 'films', 'fields': {'title': 'Star Trek Into Darkness', 'overview': 'When the crew of the Enterprise is called back home, they find an unstoppable force of terror from within their own organization has detonated the fleet and everything it stands for, leaving our world in a state of crisis.  With a personal score to settle, Captain Kirk leads a manhunt to a war-zone world to capture a one man weapon of mass destruction. As our heroes are propelled into an epic chess game of life and death, love will be challenged, friendships will be torn apart, and sacrifices mu